In [14]:
import os
import collections
import numpy as np
import tensorflow as tf

from importlib import reload
from collections import Counter
 


def make_Dictionary(train_dir):
    emails = [os.path.join(train_dir,f) for f in os.listdir(train_dir)]    
    all_words = []       
    for mail in emails:    
        with open(mail) as m:
            for i,line in enumerate(m):
                if i == 2:  #Body of email is only 3rd line of text file
                    words = line.split()
                    all_words += words
    
    dictionary = collections.Counter(all_words)
    list_to_remove = list(dictionary)
    for item in list_to_remove:
        if item.isalpha() == False: 
            del dictionary[item]
        elif len(item) == 1:
            del dictionary[item]
    dictionary = dictionary.most_common(300) 
    return dictionary




def extract_features(mail_dir, dictionary): 
    files = [os.path.join(mail_dir,fi) for fi in os.listdir(mail_dir)]
    features_matrix = np.zeros((len(files),300))
    docID = 0;
 
    for fil in files:
        with open(fil) as fi:
            for i, line in enumerate(fi):
                if i==2:
                    words=line.split()
                    for word in words:
                        wordID=0
                        for i,d in enumerate(dictionary):
                            if d[0]==word:
                                wordID=i
                                features_matrix[docID,wordID] = words.count(word)
                            
        docID = docID+1
    return features_matrix
 
    
train_dir = '/home/jovi/ling-spam/train-mails'

dictionary = make_Dictionary(train_dir)

# Prepare feature vectors per training mail and its labels

train_labels = np.zeros(702)
train_labels[351:701] = 1
train_matrix = extract_features(train_dir, dictionary)

    
# Build 3 layer DNN with 10, 20, 10 units respectively.
feature_columns = [tf.feature_column.numeric_column("x", shape=[300])]
classifier = tf.estimator.DNNClassifier(feature_columns=feature_columns,
                                          hidden_units=[10, 20, 10],
                                          n_classes=2)
# Define the training inputs
train_input_fn = tf.estimator.inputs.numpy_input_fn(
      x={"x": tf.to_int32(train_matrix, name='ToInt32')},
      y=tf.to_int32(train_labels, name='ToInt32'),
      num_epochs=None,
      shuffle=True)

# Train model.
classifier.train(input_fn=train_input_fn, steps=2000)

# Define the test inputs
# Test the unseen mails for Spam
test_dir = '/home/jovi/ling-spam/test-mails'
test_matrix = extract_features(test_dir, dictionary)
test_labels = np.zeros(260)
test_labels[130:260] = 1
test_input_fn = tf.estimator.inputs.numpy_input_fn(
  x={"x": test_matrix},
  y=tf.to_int32(test_labels, name='ToInt32'),
  num_epochs=1,
  shuffle=False)

# Evaluate accuracy.
accuracy_score = classifier.evaluate(input_fn=test_input_fn)["accuracy"]

print("\nTest Accuracy: {0:f}\n".format(accuracy_score))



INFO:tensorflow:Using default config.
INFO:tensorflow:Using config: {'_task_id': 0, '_log_step_count_steps': 100, '_num_ps_replicas': 0, '_task_type': 'worker', '_session_config': None, '_save_checkpoints_secs': 600, '_save_checkpoints_steps': None, '_keep_checkpoint_max': 5, '_save_summary_steps': 100, '_master': '', '_keep_checkpoint_every_n_hours': 10000, '_service': None, '_tf_random_seed': None, '_model_dir': '/tmp/tmpue87em5_', '_num_worker_replicas': 1, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7f217e761da0>, '_is_chief': True}


TypeError: unhashable type: 'Dimension'